# Data Merging - Scopus and SciVal

This notebook merges Scopus and SciVal datasets using the EID column.

## Objectives:
1. Load both datasets
2. Merge using EID as key
3. Add abstracts from Scopus to SciVal entries
4. Resolve duplicate columns
5. Save merged dataset

In [ ]:
import sys
sys.path.append('../')

import pandas as pd
import numpy as np
from pathlib import Path

from src.data.load_data import load_scopus_data, load_scival_data
from src.data.merge_data import (
    merge_datasets,
    add_abstracts_to_scival,
    resolve_duplicate_columns
)
from src.utils.config import config

pd.set_option('display.max_columns', None)

## 1. Load Data

In [ ]:
# Load data files
SCOPUS_FILE = '../data/raw/scopus.csv'
SCIVAL_FILE = '../data/raw/scival.csv'

# Load datasets
scopus_df = load_scopus_data(SCOPUS_FILE)
scival_df = load_scival_data(SCIVAL_FILE)

print(f"Scopus: {scopus_df.shape}")
print(f"SciVal: {scival_df.shape}")

## 2. Merge Datasets and Replace Abstracts

SciVal's Abstract column contains URLs, not actual text. We need to replace it with real abstracts from Scopus.

In [ ]:
scopus_abstracts = scopus_df[['EID', 'Abstract']].copy()
scopus_abstracts.columns = ['EID', 'Abstract_scopus']

merged_df = scival_df.merge(scopus_abstracts, on='EID', how='inner')

merged_df = merged_df.drop('Abstract', axis=1)
merged_df = merged_df.rename(columns={'Abstract_scopus': 'Abstract'})

print(f"Merged dataset shape: {merged_df.shape}")
print(f"Abstracts matched: {merged_df['Abstract'].notna().sum()} / {len(merged_df)}")

In [ ]:
# Full merge of both datasets
# merged_df = merge_datasets(
#     scopus_df=scopus_df,
#     scival_df=scival_df,
#     on='EID',
#     how='inner'  # or 'outer' to keep all records
# )

# print(f"Merged dataset shape: {merged_df.shape}")
# print(f"Columns: {len(merged_df.columns)}")

In [ ]:
# Check for duplicate columns
# duplicate_cols = [col for col in merged_df.columns if '_scopus' in col or '_scival' in col]
# print(f"Duplicate columns found: {len(duplicate_cols)}")
# if duplicate_cols:
#     print(duplicate_cols[:10])  # Show first 10

## 5. Examine Merged Data

In [ ]:
print(f"Abstracts present: {merged_df['Abstract'].notna().sum()} / {len(merged_df)}")
print(f"Missing abstracts: {merged_df['Abstract'].isna().sum()}")
print(f"\nSample abstract:\n{merged_df['Abstract'].iloc[0][:200]}...")

## 6. Save Merged Dataset

## 3. Verify Merged Data

## Summary Statistics

In [ ]:
print("=" * 50)
print("MERGE SUMMARY")
print("=" * 50)
print(f"Total records: {len(merged_df)}")
print(f"Total columns: {len(merged_df.columns)}")
print(f"Memory usage: {merged_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"\nKey columns present:")
key_cols = ['EID', 'Abstract', 'Title', 'Citations', 'Authors', 'Year']
for col in key_cols:
    if col in merged_df.columns:
        print(f"  ✓ {col}")
    else:
        print(f"  ✗ {col}")